In [ ]:
import tkinter as tk
import numpy as np
import sounddevice as sd
from consts import * 

def timeToFrame(t):
    return round(t * SRATE)

class Env:
    # points = [(t0, v0), (t1, v1), (t2, v2), ...]
    # xAxis puede ser 'time' (segundos) o 'samples'
    def __init__(self, points, xAxis='samples'):
        if points[0][0] != 0:
            raise Exception(f'Bad defined env: initial point {points[0]}')
        # Si los valores del eje x están en tiempo, los convertimos a samples
        if xAxis == 'time':
            points = [(timeToFrame(x), y) for (x, y) in points]

        # Construimos la envolvente generando segmentos entre los puntos
        self.env = np.zeros(0)
        (x0, y0) = points[0]  # Punto inicial
        for (x, y) in points[1:]:
            self.env = np.concatenate((
                self.env,
                np.linspace(y0, y, x - x0 + 1)[:-1]  # Evitamos duplicar el último valor
            ))
            (x0, y0) = (x, y)

        # Extendemos la envolvente hasta completar el último chunk
        restFrames = CHUNK - (points[-1][0] - points[0][0]) % CHUNK
        self.env = np.concatenate((self.env, np.full(restFrames, points[-1][1])))

        self.frame = 0  # Frame actual
        # Valor final de la envolvente (se mantiene indefinidamente)
        self.last = self.env[-1]

    # Devuelve el siguiente chunk de la envolvente (o un valor constante)
    def next(self):
        if self.frame < self.env.shape[0]:
            out = self.env[self.frame:self.frame + CHUNK]
            self.frame += CHUNK
            return out
        else:
            return np.array([self.last])

    # Resetea la envolvente (por ejemplo, para volver a disparar la nota)
    def reset(self):
        self.frame = 0

class Env:
    # points = [(t0, v0), (t1, v1), (t2, v2), ...]
    # xAxis puede ser 'time' (segundos) o 'samples'
    def __init__(self, points, xAxis='samples'):
        if points[0][0] != 0:
            raise Exception(f'Bad defined env: initial point {points[0]}')
        # Si los valores del eje x están en tiempo, los convertimos a samples
        if xAxis == 'time':
            points = [(timeToFrame(x), y) for (x, y) in points]

        # Construimos la envolvente generando segmentos entre los puntos
        self.env = np.zeros(0)
        (x0, y0) = points[0]  # Punto inicial
        for (x, y) in points[1:]:
            self.env = np.concatenate((
                self.env,
                np.linspace(y0, y, x - x0 + 1)[:-1]  # Evitamos duplicar el último valor
            ))
            (x0, y0) = (x, y)

        # Extendemos la envolvente hasta completar el último chunk
        restFrames = CHUNK - (points[-1][0] - points[0][0]) % CHUNK
        self.env = np.concatenate((self.env, np.full(restFrames, points[-1][1])))

        self.frame = 0  # Frame actual
        # Valor final de la envolvente (se mantiene indefinidamente)
        self.last = self.env[-1]

    # Devuelve el siguiente chunk de la envolvente (o un valor constante)
    def next(self):
        if self.frame < self.env.shape[0]:
            out = self.env[self.frame:self.frame + CHUNK]
            self.frame += CHUNK
            return out
        else:
            return np.array([self.last])

    # Resetea la envolvente (por ejemplo, para volver a disparar la nota)
    def reset(self):
        self.frame = 0


class Osc:
    def __init__(self, freq=440.0, amp=1.0, phase=0.0):
        self.freq = freq
        self.amp = amp
        self.phase = phase
        self.frame = 0

    def next(self):
        t = np.arange(self.frame, self.frame + CHUNK)
        out = self.amp * np.sin(2 * np.pi * t * self.freq / SRATE + self.phase)
        self.frame += CHUNK
        return out

class Note:
    def __init__(self, freq=440.0, attack=0.1, release=0.9, amp=0.8):
        self.osc = Osc(freq)
        # Definimos la envolvente con dos tramos:
        # - Ataque: desde 0 hasta la amplitud deseada en "attack" segundos
        # - Decaimiento: desde la amplitud hasta 0 en "release" segundos
        self.env = Env([(0, 0), (attack, amp), (attack + release, 0)], xAxis='time')

    def next(self):
        env_chunk = self.env.next()
        osc_chunk = self.osc.next()
        return osc_chunk * env_chunk



# ---------------------------INICIO DE NUESTRO CODIGO---------------------------


buffer_audio = None # como metemos al buffer audio todo un bloque de 
buffer_idx = 0

def callback(outdata, frames, time, status):
    global buffer_audio, buffer_idx
    # si hay audio en el buffer extraemos el siguiente bloque
    if buffer_audio is not None:
        s = buffer_audio[buffer_idx:buffer_idx+CHUNK] # seleccion del siguiente chunk
        buffer_idx += CHUNK
    else:
        s = np.zeros(CHUNK, dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)

stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()

def generar_notas_afinacion(base=440.0, tipo='temperada'): #genera un arry de frecuencias dependiendo de la afinacion

    escalas = {
        'justa': [1, 9/8, 5/4, 4/3, 3/2, 5/3, 15/8, 2],
        'temperada': [2**(i/12) for i in [0, 2, 3, 5, 7, 8, 10, 12]]
    }
    
    nombres = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    ratios = escalas.get(tipo, escalas['temperada'])
    
    return {nombre: base * r for nombre, r in zip(nombres, ratios)}


def generar_acorde(mapa_notas, nota): # con el array de fecuencias mapa_notas generamos un array con el acorde de la nota 

    escala = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    indice = escala.index(nota)
    offsets = [0, 2, 4]  # tónica, tercera mayor, quinta justa

    acorde = []
    for offset in offsets:
        nuevo_indice = indice + offset
        nota_actual = escala[nuevo_indice % len(escala)]
        frecuencia = mapa_notas[nota_actual] * (2 if nuevo_indice >= len(escala) else 1) # si el indice se sale de la len(escala) se mul por dos
        acorde.append(frecuencia)
    
    return acorde


notas_justa = generar_notas_afinacion(440.0, 'justa')
notas_temperada = generar_notas_afinacion(440.0, 'temperada')

notas = { # generamos mapa de notas
    'q': ('A', generar_acorde(notas_justa, 'A')),
    'w': ('B', generar_acorde(notas_justa, 'B')),
    'e': ('C', generar_acorde(notas_justa, 'C')),
    'r': ('D', generar_acorde(notas_justa, 'D')),
    't': ('E', generar_acorde(notas_justa, 'E')),
    'y': ('F', generar_acorde(notas_justa, 'F')),
    'u': ('G', generar_acorde(notas_justa, 'G')),

    'z': ('A', generar_acorde(notas_temperada, 'A')),
    'x': ('B', generar_acorde(notas_temperada, 'B')),
    'c': ('C', generar_acorde(notas_temperada, 'C')),
    'v': ('D', generar_acorde(notas_temperada, 'D')),
    'b': ('E', generar_acorde(notas_temperada, 'E')),
    'n': ('F', generar_acorde(notas_temperada, 'F')),
    'm': ('G', generar_acorde(notas_temperada, 'G'))
}


def play_chord(key):
    note_name, freqs = notas[key]
    notes_list = [Note(freq, attack=0.1, release=0.9, amp=0.8) for freq in freqs] #generamos un array con todas las notas

    total_duration = 1
    total_frames = timeToFrame(total_duration)
    if total_frames % CHUNK != 0:
        total_frames += CHUNK - (total_frames % CHUNK) # calculamos el tamaño del ultimo chunk

    output = np.array([], dtype=np.float32)
    
    frames_generated = 0
    while frames_generated < total_frames:
        chunk_sum = np.zeros(CHUNK)
        for note in notes_list:
            chunk_sum += note.next() #generamos cada .next de cada nota del acorde
        output = np.concatenate((output, chunk_sum))
        frames_generated += CHUNK

    # Normalización de la amplitud para evitar distorsion
    max_val = np.max(np.abs(output))
    if max_val > 0:
        output = output / max_val

    return output

def on_key(event):
    global buffer_audio, buffer_idx
    if event.char in notas:
        buffer_audio = play_chord(event.char)
        buffer_idx = 0
    else:
        buffer_audio = None

# Creamos la ventana principal
root = tk.Tk()
root.title("Generador de acordes")
root.geometry("400x200")
root.bind("<KeyPress>", on_key)
label = tk.Label(root, text="Presiona zxcvbnm para afinacion temperada y qwertyu para la justa", font=("Arial", 12))
label.pack(pady=20)

root.mainloop()



Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7fe77c374c20>:
Traceback (most recent call last):
  File "/home/llvch/Desktop/uni/.venv/lib/python3.12/site-packages/sounddevice.py", line 873, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/llvch/Desktop/uni/.venv/lib/python3.12/site-packages/sounddevice.py", line 2735, in _wrap_callback
    callback(*args)
  File "/tmp/ipykernel_39972/3808711608.py", line 134, in callback
ValueError: could not broadcast input array from shape (0,1) into shape (1024,32)
